In [41]:
# Clustering Toronto Neighbourhoods into 4 distincs clusters (Central, Down-Town, East & West Toronto) and looking at the venues in Central Toronoto using Foursqaure APIs"

In [42]:
import requests # import & load  necessary libraries and packages
import lxml.html as lh
import pandas as pd

In [43]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'  # wikipedia URL tnat has the Toronto Postcodes and details

In [44]:
from bs4 import BeautifulSoup  # import Beautifulsoup # Get BeautifulSoup package
print('BeautifulSoap imported')

BeautifulSoap imported


In [45]:
# Using BS to scrap the wikipedia HTML and generate a csv file with the postal codes etc
import requests  # import requests library

In [46]:
website_URL=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [47]:
Toronto_soup = BeautifulSoup(website_URL,'lxml')  # Align it to Toronto

In [48]:
My_table = Toronto_soup.find('table',class_='wikitable sortable') # obtain wikitable sortable

In [49]:
#Create a handle, page, to handle the contents of the website
page = requests.get(url)

In [50]:
#Store the contents of the website under doc
doc = lh.fromstring(page.content)

In [51]:
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [52]:
#Check the length of the first 12 rows
    [len(T) for T in tr_elements[:12]]    

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [53]:
tr_elements = doc.xpath('//tr')
#Create empty list
Dict=[]
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content().strip("\n")
    print ('%d:%s'%(i,name))
    col.append((name,[]))

1:Postcode
2:Borough
3:Neighbourhood


In [54]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [55]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [56]:
Dict={title:column for (title,column) in col}   # Dictionary with Postcodes, Borough and Neighbourhoods
df=pd.DataFrame(Dict)  # Creating the pandas Dataframe using the dictionary values

In [57]:
#df

In [58]:
df_Borough=df[df.Borough != 'Not assigned']   # get rid of "Not assigned" boroughs

In [59]:
#df_Borough

In [60]:
import numpy as np
df_Borough.columns = ['Postcode', 'Borough', 'Neighbourhood']

In [61]:
##df_Borough

In [62]:
df_Borough['Neighbourhood'] = df_Borough['Neighbourhood'].astype(str).str[:-1].astype(str)  # get rid of the eol marker from the neighbourhood values
#df_Borough['Neighbourhood']=df_Borough['Neighbourhood'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True) 

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [63]:
#df_Borough

In [64]:
mask = df_Borough.Neighbourhood == 'Not assigned'  # assign Borough value to those neighbourhoods with "Not assigned"
column_name = 'Neighbourhood'
df_Borough.loc[mask, column_name] = df_Borough.Borough

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [65]:
#df_Borough

In [66]:

df_new=df_Borough.groupby('Postcode', sort=False)['Neighbourhood'].apply(','.join)  # group neighbourhoods by Postcode

In [67]:
#df_new

In [68]:

df_new_FINAL=pd.DataFrame({'Postcode':df_new.index, 'df_new':df_new.values})
df_new_F=df_new_FINAL.rename(columns={ df_new_FINAL.columns[1]: "Neighbourhood" })
df_N=df_new_F['Neighbourhood'].tolist() # create a neighbourhood list

In [69]:
#df_new

In [70]:

df_Borough_BONLY=df_Borough['Borough'] # create a Borough list
df_Borough_PONLY=df_Borough['Postcode'] # create a postcode list

In [71]:
#df_Borough_BONLY

In [72]:
#df_Borough_PONLY

In [73]:
df_Borough_BandP=pd.concat([df_Borough_PONLY, df_Borough_BONLY], axis=1)

In [74]:
#df_Borough_BandP

In [75]:
df_Borough_BandP_new = df_Borough_BandP.drop_duplicates(subset=['Postcode','Borough'])  # drop duplicates

In [76]:
#df_P
#df_B

In [77]:
#df_Borough_BandP_new

In [78]:
df_P=df_Borough_BandP_new['Postcode'].tolist() # refined lists
df_B=df_Borough_BandP_new['Borough'].tolist() # refined lists

In [79]:
Toronto_DICT= {'Postcode':df_P,'Borough':df_B, 'Neighbourhood':df_N} # Create a dictionary with the 3 refined lists
Toronto_df=pd.DataFrame(Toronto_DICT) # create the dataframe

In [80]:
# List of Postcodes, Boroughs, & Neighbourhoods in TORONTO FOR 103 POSTCODES
Toronto_df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [82]:
Toronto_df.shape

(103, 3)